# JupyterHub Spawners

Let's peek at the base classes:

In [1]:
from jupyterhub.spawner import Spawner, LocalProcessSpawner

In [2]:
Spawner??

Init signature: Spawner(self, **kwargs)
Source:
class Spawner(LoggingConfigurable):
    """Base class for spawning single-user notebook servers.
    
    Subclass this, and override the following methods:
    
    - load_state
    - get_state
    - start
    - stop
    - poll
    """
    
    db = Any()
    user = Any()
    hub = Any()
    authenticator = Any()
    api_token = Unicode()
    ip = Unicode('127.0.0.1',
        help="The IP address (or hostname) the single-user server should listen on"
    ).tag(config=True)
    start_timeout = Integer(60,
        help="""Timeout (in seconds) before giving up on the spawner.
        
        This is the timeout for start to return, not the timeout for the server to respond.
        Callers of spawner.start will assume that startup has failed if it takes longer than this.
        start should return when the server process is started and its location is known.
        """
    ).tag(config=True)

    http_timeout = Integer(30,
        help="

Start is the key method in a Spawner. It's how we decide how to start the process that will become the single-user server:

In [3]:
LocalProcessSpawner.start??

Signature: LocalProcessSpawner.start(self)
Source:
    @gen.coroutine
    def start(self):
        """Start the process"""
        if self.ip:
            self.user.server.ip = self.ip
        self.user.server.port = random_port()
        cmd = []
        env = self.get_env()
        
        cmd.extend(self.cmd)
        cmd.extend(self.get_args())
        
        self.log.info("Spawning %s", ' '.join(pipes.quote(s) for s in cmd))
        self.proc = Popen(cmd, env=env,
            preexec_fn=self.make_preexec_fn(self.user.name),
            start_new_session=True, # don't forward signals
        )
        self.pid = self.proc.pid

File:      ~/conda/envs/jupyterhub-tutorial/lib/python3.5/site-packages/jupyterhub/spawner.py
Type:      function


Here is an example of a spawner that allows specifying extra *arguments* to pass to a user's notebook server, via `.options_form`. It results in a form like this:

![form](img/spawn-form.png)

In [4]:
from traitlets import default

class DemoFormSpawner(LocalProcessSpawner):
    @default('options_form')
    def _options_form(self):
        default_env = "YOURNAME=%s\n" % self.user.name
        return """
        <label for="args">Extra notebook CLI arguments</label>
        <input name="args" placeholder="e.g. --debug"></input>
        """.format(env=default_env)
    
    def options_from_form(self, formdata):
        """Turn html formdata (always lists of strings) into the dict we want."""
        options = {}
        arg_s = formdata.get('args', [''])[0].strip()
        if arg_s:
            options['argv'] = shlex.split(arg_s)
        return options
    
    def get_args(self):
        """Return arguments to pass to the notebook server"""
        argv = super().get_args()
        if self.user_options.get('argv'):
            argv.extend(self.user_options['argv'])
        return argv
    
    def get_env(self):
        """Return environment variable dict"""
        env = super().get_env()
        return env

## Exercise:

Write a custom Spawner that allows users to specify *environment variables* to load into their server.


-------

In [5]:
from dockerspawner import DockerSpawner

In [6]:
DockerSpawner.start??

Signature: DockerSpawner.start(self, image=None, extra_create_kwargs=None, extra_start_kwargs=None, extra_host_config=None)
Source:
    @gen.coroutine
    def start(self, image=None, extra_create_kwargs=None,
        extra_start_kwargs=None, extra_host_config=None):
        """Start the single-user server in a docker container. You can override
        the default parameters passed to `create_container` through the
        `extra_create_kwargs` dictionary and passed to `start` through the
        `extra_start_kwargs` dictionary.  You can also override the
        'host_config' parameter passed to `create_container` through the
        `extra_host_config` dictionary.

        Per-instance `extra_create_kwargs`, `extra_start_kwargs`, and
        `extra_host_config` take precedence over their global counterparts.

        """
        container = yield self.get_container()
        if container is None:
            image = image or self.container_image

            # build the dictionary of

## Exercise:

Subclass DockerSpawner so that users can specify via `options_form` what docker image to use.

Candidates from the [Jupyter docker-stacks repo](https://github.com/jupyter/docker-stacks) include:

- jupyter/minimal-singleuser
- jupyter/scipy-singleuser
- jupyter/r-singleuser
- jupyter/datascience-singleuser
- jupyter/pyspark-singleuser

Or, build your own images with

    FROM jupyterhub/singleuser
    
The easiest version will assume that the images are fetched already.

## Extra credit:

Subclass DockerSpawner so that users can specify via `options_form` a GitHub repository to clone and install, a la [binder](http://mybinder.org).